# Final Project

## Data loading and preprocessing

# Final Project CS5680

## Loading Data and Preprocessing

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import shutil
import matplotlib.pyplot as plt
import os
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.metrics import confusion_matrix
import pandas as pd

### Programatically Making the Classes
The JAFFE dataset did not have the classes made for me. I had to programatically go through and add them to a class. This code does just that. This code is dependent on your file structure and will not work for anyone without the correct file structure

In [ ]:
# programatically make the classes
image_directory = 'JAFFE/jaffedbase/jaffedbase'
happy_directory = 'JAFFE/happy'
angry_directory = 'JAFFE/angry'
sad_directory = 'JAFFE/sad'
surprised_directory = 'JAFFE/surprised'
disgusted_directory = 'JAFFE/disgusted'
fearful_directory = 'JAFFE/fearful'
neutral_directory = 'JAFFE/neutral'

for directory in [happy_directory, angry_directory, sad_directory, surprised_directory, disgusted_directory, fearful_directory, neutral_directory]:
    if not os.path.exists(directory):
        os.makedirs(directory)

for filename in os.listdir(image_directory):
    if 'HA' in filename:
        source_path = image_directory + '/' + filename
        destination_path = happy_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'AN' in filename:
        source_path = image_directory + '/' + filename
        destination_path = angry_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'SA' in filename:
        source_path = image_directory + '/' + filename
        destination_path = sad_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'SU' in filename:
        source_path = image_directory + '/' + filename
        destination_path = surprised_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'DI' in filename:
        source_path = image_directory + '/' + filename
        destination_path = disgusted_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'FE' in filename:
        source_path = image_directory + '/' + filename
        destination_path = fearful_directory + '/' + filename
        shutil.move(source_path, destination_path)
    elif 'NE' in filename:
        source_path = image_directory + '/' + filename
        destination_path = neutral_directory + '/' + filename
        shutil.move(source_path, destination_path)


#### Convert .tiff to .png
This step of preprocessing was necessary because tensorflow's image_dataset_from_directory relies on it being a certain format.

The output of this contained an error because I was looking at the file. I just removed this .tiff file.

In [ ]:
# Path to the directory containing your TIFF images
classes = ['happy', 'angry', 'sad', 'surprised', 'disgusted', 'fearful', 'neutral']
for class_name in classes:
    tiff_directory = 'JAFFE/' + class_name

    # # Path to the directory where you want to save the PNG images
    png_directory = 'JAFFE/' + class_name

    # Create the PNG directory if it doesn't exist
    os.makedirs(png_directory, exist_ok=True)

    # Iterate through each TIFF file in the directory
    for tiff_filename in os.listdir(tiff_directory):
        if tiff_filename.endswith('.tiff') or tiff_filename.endswith('.tif'):
            tiff_path = os.path.join(tiff_directory, tiff_filename)

            # Open the TIFF image using Pillow
            tiff_image = Image.open(tiff_path)

            # Convert and save the image as PNG
            png_filename = os.path.splitext(tiff_filename)[0] + '.png'
            png_path = os.path.join(png_directory, png_filename)
            tiff_image.save(png_path, format='PNG')
            tiff_image.close()
            os.remove(tiff_path)

Loading the dataset from a directory

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('JAFFE', color_mode='grayscale', image_size=(256, 256), batch_size=4, shuffle=True) #change batch size after testing
data_iterator = data.as_numpy_iterator()
data_batch = next(data_iterator)

### Get the face and resize image
In the paper, they got the face and resized the image. The following commented out code was my original attempt. However, I was spending too much time on it, so I just continued without it. From here, I can either get this implementation working, or I can just manually slice out the needed pixels. I could do this programatically because each of the images are relatively similar. The face is always in the same rough position.

Regardless, this WILL be completed before I move too far onwards.

In [ ]:
# get the face and resize the image
def segment_face(data_batch):
    top, bottom, left, right = 16, 16, 48, 48

    height, width, channels = data_batch[0].shape

    face_images = []

    for image in data_batch:
        segmented_image = np.array(image[top:height-bottom, left:width-right])
        segmented_image = cv2.UMat(segmented_image)
        face_image = cv2.resize(segmented_image, (128, 128))
        face_image = cv2.UMat.get(face_image)
        face_images.append(face_image)

    return tf.convert_to_tensor(face_images)

data_segmented = data.map(lambda x, y: (tf.py_function(segment_face, [x], [tf.float32]), y))
data_segmented_iterator = data_segmented.as_numpy_iterator()
data_segmented_batch = next(data_segmented_iterator)

#### Piecing out the image
In the paper, after they found the face, they pieced out the image in order to continue with their method. This is what this section of code is doing.

In [ ]:
# divide each image into pieces
piece_size = 32
def divide_image(image):
    divided_images = []
    for k in range(0, image.shape[0]):
        batch_image_divided = []
        for i in range(0, image.shape[1], piece_size):
            for j in range(0, image.shape[2], piece_size):
                divided_image = image[k, i:i+piece_size, j:j+piece_size]
                batch_image_divided.append(divided_image)
        divided_images.append(batch_image_divided)
    return tf.convert_to_tensor(divided_images)

data_divided = data_segmented.map(lambda x, y: (tf.py_function(divide_image, [x[0]], [tf.float32]), y))
data_divided_iterator = data_divided.as_numpy_iterator()
data_divided_batch = next(data_divided_iterator)

#### The proposed texture based transformation
In the paper, they perform 5 levels of graph based texture transformation. So far, the below code has only done one. The graph based texture transformation utilizes a "sigmoid" function. This function takes two parameters, a and b, and returns 0 if a is less than b, otherwise it returns 1. The parameter "a" is the pixel that you are "on"/looking at. It gets compared with the parameter b, which is the pixel you are comparing to. In the first level graph based texture transformation, you utilize two graphs. You also utilize a kernel size, for mine, I just did a 3x3 kernel/window. The first graph takes the middle pixel in the window (the start pixel/pixel you are looking at is in the top left) and compares it to each of the other pixels. It first compares it with the top left, then top middle, then top right, then right, so on and so forth in a circle/square. This binary coding is then preserved. The second graph takes the top left pixel, or the start, and compares it to its right neighbor. The right neighbor is then compared with its right neighbor. That neighbor is compared with the neighbor below it. This graph essentially starts that the top left, and makes a comparison in a square like formation. The paper has a very great visual on this for more information. 

This is the first major research component in the paper. This component is supposed to be very helpful when removing necessary features. It is worth noting that in the paper, they tested 3x3, 5x5, and 7x7 graphs, in my implementation, I have only used a 3x3. This could be a potential change in my method that may help or decrease my model's accuracy.

In [ ]:
# The propose texture based transformation
#a is the one youre on, b is the one youre comparing to
def sigmoid(a, b):
    return int(a<b)

##### First Level Graph Based Texture Transformation

This code was for testing. It should not be run in production unless desired.

In [ ]:
def graph_based_level_one_transformation(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []
    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        for c in range(0, image_array.shape[1]): # for each division of the image
            binary_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    features_one = [
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+2]),
                    ]

                    features_two = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j])         
                    ]

                    binary_features.append(features_one+features_two)
            combined_binary_features = tf.concat([tf.convert_to_tensor(binary_features, dtype=tf.float32)], axis=-1)
        batch_binary_features.append(combined_binary_features)
    tf.cast(batch_binary_features, tf.float32)
    return tf.convert_to_tensor(batch_binary_features)

This code was for testing. It should not be run in production unless desired.

In [ ]:
level_one_binary_features = data_divided.map(lambda x, y: (tf.py_function(graph_based_level_one_transformation, [x[0]], tf.float32), y))
level_one_binary_features_iterator = level_one_binary_features.as_numpy_iterator()
try:
    level_one_binary_features_batch = next(level_one_binary_features_iterator)
except StopIteration:
    print("End of iterator reached.")
except Exception as e:
    print("An error occurred:", e)
level_one_binary_features_batch

# End of Checkpoint 1 Comments
At this point in the implementation, I have only completed data loading and preprocessing. There was some difficulties with recognizing the face and segmenting the image. I will end up going back and fixing this by next checkpoint. 

This checkpoint, I completed my data loading and preprocessing (for the most part). I also started on one of, if not the most, major aspect of this paper/implementation. By next checkpoint, I hope to have completed the graph based texture transformation and moved onto the next major aspect. After that major aspect, the only thing left is to train the model and measure it's performance.

##### Second Level Graph Based Texture Transformation

In the second level graph based texture transformation, you utilize four graphs. I continued to utilize a 3x3 kernel/window. All of these graphs are slight modification of the first graph. The first graph is in the shape of a pentagon. It starts that the top pixel (the point of the pentagon). It compares this pixiel with the next one on the pentagon (down and left). It continues around the pentagon until it reaches the top pixel again. The secondg graph is the same as the first, except the pentagon is rotated -90 or 270 degrees. The tip is now at the left pixel. The third graph is the same as the first, except the pentagon is rotated 180 or -180 degrees. The tip is now at the bottom pixel. The fourth graph is the same as the first, except the pentagon is rotated 90 or -270 degrees. The tip is now at the right pixel. 

The actual operations are the same as the first level graph based texture transformation. It utilizes the sigmoid function from above. The main difference between these two levels is the shape of the graphs and the number of binary features it extracts. This level extracts 20 binary features. The paper has a very great visual on this for more information.

This code was for testing. It should not be run in production unless desired.

In [ ]:
def graph_based_level_two_transformation(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []
    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        for c in range(0, image_array.shape[1]): # for each division of the image
            binary_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    features_one = [
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j+1]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                    ]
                
                    features_four = [
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j+2]),
                    ]

                    binary_features.append(features_one+features_two+features_three+features_four)
            combined_binary_features = tf.concat([tf.convert_to_tensor(binary_features, dtype=tf.float32)], axis=-1)
        batch_binary_features.append(combined_binary_features)
    tf.cast(batch_binary_features, tf.float32)
    return tf.convert_to_tensor(batch_binary_features)


This code was for testing. It should not be run in production unless desired.

In [ ]:
level_two_binary_features = data_divided.map(lambda x, y: (tf.py_function(graph_based_level_two_transformation, [x[0]], tf.float32), y))
# level_two_binary_features = level_two_binary_features.map(lambda x, y: (tf.convert_to_tensor(x), y))
level_two_binary_features_iterator = level_two_binary_features.as_numpy_iterator()
try:
    level_two_binary_features_batch = next(level_two_binary_features_iterator)
except StopIteration:
    print("End of iterator reached.")
except Exception as e:
    print("An error occurred:", e)
level_two_binary_features_batch


##### Third Level Graph Based Texture Transformation

In the third level graph based texture transformation, you utilize two graphs. I continued to utilize a 3x3 kernel/window. The first graph of this level is in a diamond shape. It starts at the left pixel. This pixel is compared to the pixel to the right of it (top pixel). It follows this order all the way around the diamond. This will result in 4 binary features. The second graph starts at the center pixel. It compares it to all of the diagonals. This will result in 4 binary features. Once again, the paper has a very great visual on this for more information. This will result in 8 binary features.


This code was for testing. It should not be run in production unless desired.

In [ ]:
def graph_based_level_three_transformation(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []
    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        for c in range(0, image_array.shape[1]): # for each division of the image
            binary_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    features_one = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j]),
                    ]

                    binary_features.append(features_one+features_two)
            combined_binary_features = tf.concat([tf.convert_to_tensor(binary_features, dtype=tf.float32)], axis=-1)
        batch_binary_features.append(combined_binary_features)
    tf.cast(batch_binary_features, tf.float32)
    return tf.convert_to_tensor(batch_binary_features)

This code was for testing. It should not be run in production unless desired.

In [ ]:
level_three_binary_features = data_divided.map(lambda x, y: (tf.py_function(graph_based_level_three_transformation, [x[0]], tf.float32), y))
# level_three_binary_features = level_three_binary_features.map(lambda x, y: (tf.convert_to_tensor(x), y))
level_three_binary_features_iterator = level_three_binary_features.as_numpy_iterator()
try:
    level_three_binary_features_batch = next(level_three_binary_features_iterator)
except StopIteration:
    print("End of iterator reached.")
except Exception as e:
    print("An error occurred:", e)
level_three_binary_features_batch

##### Fourth Level Graph Based Texture Transformation

In the fourth level graph based texture transformation, you utilize four graphs. Once again, I utilized a 3x3 kernel size. The first graph is in the shape of a triangle. It compares the top pixel with the bottom right pixel. It then compares the bottom right pixel with the bottom left pixel. This will result in 3 binary features being extracted. The second graph is an altered version of the first. This graph is still the triangle shape, but it is rotated -90 or 270 degrees. The third graph is an altered version of the first. This graph is still the triangle shape, but it is rotated 180 or -180 degrees. The fourth graph is an altered version of the first. This graph is still the triangle shape, but it is rotated 90 or -270 degrees. This will result in 12 binary features being extracted.

This code was for testing. It should not be run in production unless desired.

In [ ]:
def graph_based_level_four_transformation(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []
    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        for c in range(0, image_array.shape[1]): # for each division of the image
            binary_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    features_one = [
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+2, j]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i, j]),
                    ]
                
                    features_four = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j]),
                    ]

                    binary_features.append(features_one+features_two+features_three+features_four)
            combined_binary_features = tf.concat([tf.convert_to_tensor(binary_features, dtype=tf.float32)], axis=-1)
        batch_binary_features.append(combined_binary_features)
    tf.cast(batch_binary_features, tf.float32)
    return tf.convert_to_tensor(batch_binary_features)

This code was for testing. It should not be run in production unless desired.

In [ ]:
level_four_binary_features = data_divided.map(lambda x, y: (tf.py_function(graph_based_level_four_transformation, [x[0]], tf.float32), y))
# level_four_binary_features = level_four_binary_features.map(lambda x, y: (tf.convert_to_tensor(x), y))
level_four_binary_features_iterator = level_four_binary_features.as_numpy_iterator()
try:
    level_four_binary_features_batch = next(level_four_binary_features_iterator)
except StopIteration:
    print("End of iterator reached.")
except Exception as e:
    print("An error occurred:", e)
level_four_binary_features_batch

##### Fifth Level Graph Based Texture Transformation


The fifth and final level graph based texture transformation utilizes three different graphs. The first of the three graphs compares all the pixels in a line. It compares the top left pixel with the bottom left, the top pixel with the bottom pixel, and the top right pixel with the bottom left pixel. This graph extracts 3 binary features. The second graph compares in a horizontal line. The top left pixel is compared with the top right, the left pixel is compared with the right pixel, the bottom left pixel is compared with the bottom right pixel. This graph extracts 3 binary features. The third graph compares in a diagonal line. The top left pixel is compared with the bottom right pixel, the top right pixel is compared with the bottom left pixel. This graph extracts 2 binary features. This will result in 8 binary features being extracted.

This code was for testing. It should not be run in production unless desired.

In [ ]:
def graph_based_level_five_transformation(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []
    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        for c in range(0, image_array.shape[1]): # for each division of the image
            binary_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    features_one = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+2]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j]),
                    ]

                    binary_features.append(features_one+features_two+features_three)
            combined_binary_features = tf.concat([tf.convert_to_tensor(binary_features, dtype=tf.float32)], axis=-1)
        batch_binary_features.append(combined_binary_features)
    tf.cast(batch_binary_features, tf.float32)
    return tf.convert_to_tensor(batch_binary_features)

This code was for testing. It should not be run in production unless desired.

In [ ]:
level_five_binary_features = data_divided.map(lambda x, y: (tf.py_function(graph_based_level_five_transformation, [x[0]], tf.float32), y))
# level_five_binary_features = level_five_binary_features.map(lambda x, y: (tf.convert_to_tensor(x), y))
level_five_binary_features_iterator = level_five_binary_features.as_numpy_iterator()
try:
    level_five_binary_features_batch = next(level_five_binary_features_iterator)
except StopIteration:
    print("End of iterator reached.")
except Exception as e:
    print("An error occurred:", e)
level_five_binary_features_batch

The following is the final step in graph based texture transformation. This step extracts the 64 bit features from each level. The 8 feature images are constructed using these bits.  

In [ ]:
#Incorrect and not needed with new implementation
def combine_bit_features(batch):
    batch_images = []
    for c in range(batch.shape[0]):
        image_bits = []
        for d in range(batch.shape[1]): # for each division of the image
            division_bits = []
            for i in range(batch.shape[2]): # For each 64 binary feature list
                division_bits.extend(batch[c][d][i])
            image_bits.append(division_bits)
        batch_images.append(image_bits)
    return tf.convert_to_tensor(batch_images, dtype=tf.int32)    

In [ ]:
def construct_feature_images(binary_features):
    dvk_images_batch = []
    for c in range(0, binary_features.shape[0]):
        dvk_batch_image = []
        for d in range(0, binary_features.shape[1]): #each divided image
            divided_dvk_image = []
            for k in range(0, binary_features.shape[2]): #each 64 bit feature
                dvk_feature_images = []
                for i in range(0, binary_features.shape[3], 8):
                    dvk_image = []
                    dvk = 0
                    for j in range(0, 8):
                        dvk += binary_features[c, d, k, i+j].numpy()*(2**(7-j))
                        dvk_image.append(dvk)
                    dvk_feature_images.extend(dvk_image)
                divided_dvk_image.append(tf.cast(cv2.calcHist([np.array(dvk_feature_images).astype(np.uint8)], None, None, [64], [0, 256]).reshape(-1), dtype=tf.int32)) # histogram of the dvk image, this is done in the paper
            dvk_batch_image.append(divided_dvk_image)
        dvk_images_batch.append(dvk_batch_image)
    return tf.convert_to_tensor(dvk_images_batch, dtype=tf.int32)

In [ ]:
def full_graph_based_with_combination(image):
    #i, j is top left of kernel
    # for 3x3 kernel
    batch_binary_features = []

    image_array = image
    for k in range(0, image_array.shape[0]): #for each image in the batch
        image_features = []
        for c in range(0, image_array.shape[1]): # for each division of the image
            division_features = []
            for i in range(0, image_array.shape[2] - 2, 3):
                for j in range(0, image_array.shape[3] - 2, 3):
                    binary_features = []
                    # level 1
                    features_one = [
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+2]),
                    ]

                    features_two = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j])         
                    ]
                    binary_features.extend(features_one+features_two)

                    # level 2
                    features_one = [
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j+1]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                    ]
                
                    features_four = [
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j+2]),
                    ]
                    binary_features.extend(features_one+features_two+features_three+features_four)

                    # level 3
                    features_one = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i+1, j]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+1, j+1], image_array[k, c, i+1, j]),
                    ]
                    binary_features.extend(features_one+features_two)


                    # level 4
                    features_one = [
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i, j+1]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+2, j]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i+2, j+2], image_array[k, c, i+1, j]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i+2, j+1], image_array[k, c, i, j]),
                    ]
                
                    features_four = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+1, j+2], image_array[k, c, i, j]),
                    ]
                    binary_features.extend(features_one+features_two+features_three+features_four)


                    # level 5
                    features_one = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j]),
                        sigmoid(image_array[k, c, i, j+1], image_array[k, c, i+2, j+1]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j+2]),
                    ]
                    
                    features_two = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i, j+2]),
                        sigmoid(image_array[k, c, i+1, j], image_array[k, c, i+1, j+2]),
                        sigmoid(image_array[k, c, i+2, j], image_array[k, c, i+2, j+2]),
                    ]
                    
                    features_three = [
                        sigmoid(image_array[k, c, i, j], image_array[k, c, i+2, j+2]),
                        sigmoid(image_array[k, c, i, j+2], image_array[k, c, i+2, j]),
                    ]

                    binary_features.extend(features_one+features_two+features_three)
                    division_features.append(binary_features)

            image_features.append(division_features)
        batch_binary_features.append(image_features)
    tf.cast(batch_binary_features, tf.int32)
    return tf.convert_to_tensor(batch_binary_features, dtype=tf.int32)
    # return tf.convert_to_tensor(combine_bit_features(tf.convert_to_tensor(batch_binary_features, dtype=tf.int32)), dtype=tf.int32)

In [ ]:
combined_binary_features_data = data_divided.map(lambda x, y: (tf.py_function(full_graph_based_with_combination, [x[0]], tf.int32), y))

In [ ]:
feature_images_data = combined_binary_features_data.map(lambda x, y: (tf.py_function(construct_feature_images, [x], tf.int32), y))

#### The Texture Transformation Based Facial Expression Recognition

##### 1D Maximum Pooling and PCA Feature Reduction

At this point, there are tons of features for each division of the image. This needs to be toned down. In order to decrease the dimensionality of the data, I utilized 1D maximum pooling. This is a very simple operation. It takes the maximum value in a window and sets that as the value for the window. In the paper, they utilized a 24 window size, but I utilized a 64 window size. This is due to computational complexity and for testing purposes. I will be intrigued to see if this affects the accuracy of my model. This is done in order to decrease the dimensionality of the data. The paper also utilized a PCA feature reduction. This is a very common feature reduction technique. I utilized the PCA feature reduction in order to decrease the dimensionality of the data. The paper utilized 64 features, so I utilized 64 features as well.

In [ ]:
def feature_reduction(image_batch):
    batch_pooled_features = []
    for k in range(image_batch.shape[0]):
        pooled_features = []
        for div in range(image_batch.shape[1]):
            pooled_feature = []
            # feature_reduced_divided_image = []
            #Original did 24 size of windows, but I am going to try 64. Partially for experimentation and partially to limit computation time
            divided_image_flattened = np.array(image_batch[k][div]).reshape(-1)
            for i in range(0, len(divided_image_flattened), 24):
                pooled_feature.append(max(divided_image_flattened[i:i+24])/24)
            pooled_features.append(pooled_feature)
        batch_pooled_features.append(pooled_features)
    batch_pca_features = []
    for k in range(len(batch_pooled_features)):
        division_pca_features = []
        for div in range(len(batch_pooled_features[k])):
            pooled_features_2d = np.array(batch_pooled_features[k][div]).reshape(-1, 1)
            # pca = PCA(n_components=min(pooled_features_2d.shape[0], 128))
            pca = PCA(n_components=min(pooled_features_2d.shape[0], pooled_features_2d.shape[1]))
            pooled_feature_pca = pca.fit_transform(pooled_features_2d)
            pca_features = pooled_feature_pca.flatten()
            division_pca_features.append(pca_features)
        batch_pca_features.append(division_pca_features)
    return tf.convert_to_tensor(batch_pca_features, dtype=tf.float32)


# End of Checkpoint 2 Comments
During this checkpoint, I completed the graph based texture transformation. This checkpoint, I noticed that my original implementations had some problems. Namely, the dividing image and other graph based transformations. Dividing the image was not correctly working, it was only returning one divided pieces instead of all the divided pieces. The graph based texture transformation was also only doing the one divided piece. Along with that, it was also only doing a single image in the batch. These were oversights/errors that I had to fix this checkpoint. I also started on the next major aspect of the paper, the 1D pooling and pca feature extraction. The only thing that is left is splitting the training and testing data, creating the model, and training the model. I'm hoping that this is the easiest part of the assignment. I'm also hoping that it does not take too long to train, if it does, I may have to slim down my test sets. This will affect the score of my model, but at least I will be able to score it.


### Setting up the model for training
This section will have two parts.
- The model fitted and trained on the full dataset
- The model fitted and trained on a reduced dataset (happy, neutral, and sad) (31 happy images, 30 neutral images, 31 sad images)

In [ ]:
feature_reduced_data = feature_images_data.map(lambda x, y: (tf.py_function(feature_reduction, [x], tf.float32), y))

##### Train Test Splitting  

In [ ]:
train_count = int(0.8 * len(feature_reduced_data))
test_count = int(0.2 * len(feature_reduced_data)+1)
print(train_count, test_count)

In [ ]:
train_data = feature_reduced_data.take(train_count)
test_data = feature_reduced_data.skip(train_count).take(test_count)
train_data_iterator = train_data.as_numpy_iterator()
test_data_iterator = test_data.as_numpy_iterator()

This cell is for the reduced dataset.

In [ ]:
def extract_features(data):
    features = []
    labels = []
    for batch in data:
        print(batch)
        features.append(batch[0])
        labels.append(batch[1])
    return np.array(features), np.array(labels)

This code cell will run all preprocessing steps

In [ ]:
train_feature_and_labels_extracted = extract_features(train_data)
train_features = train_feature_and_labels_extracted[0]
train_labels = train_feature_and_labels_extracted[1]

Alternatively, you can load the data from the npz file. This file was saved by me earlier, and contains all of the preprocessed data

In [ ]:
loaded_data = np.load('train_data_working.npz')
reshaped_features = loaded_data['features']
train_labels = loaded_data['labels']

This code cell reshapes the data, this is only necessary if you did not load the data in from npz file

In [ ]:
num_samples, num_channels, height, width = train_features.shape
reshaped_features = train_features.reshape(num_samples * num_channels, -1)
train_labels = train_labels.ravel()
train_labels.shape, reshaped_features.shape, type(train_labels), type(reshaped_features)

This cell saves the preprocessed training data.

In [ ]:
np.savez('train_data_working_1.npz', features=reshaped_features, labels=train_labels)

This cell utilizes a Grid Search to get the best hyperparameters

In [ ]:
svm_model = SVC()
grid_search = GridSearchCV(
    svm_model,
    param_grid={"C": np.logspace(-4, 4, 20), "gamma": np.logspace(-4, 4, 20), "kernel": ['linear', 'rbf', "poly"]},
    scoring="accuracy",
    n_jobs=-1,
    return_train_score=True,
    cv=5
)

grid_search.fit(reshaped_features, train_labels)

best_params = grid_search.best_params_
print("Best C:", best_params['C'])
print("Best gamma:", best_params['gamma'])
print("Best kernel:", best_params['kernel'])

This cell creates the model and fits it

In [ ]:
svm_model = SVC(kernel=best_params['kernel'], C=best_params['C'], gamma=best_params['gamma'])
svm_model.fit(reshaped_features, train_labels)

Cross validate the model and display the results

In [ ]:
svm_scores = cross_validate(svm_model, reshaped_features, train_labels, cv=5, scoring='accuracy', return_train_score=True)
pd.DataFrame(svm_scores)

Confusion Matrix for more visualization

In [ ]:
training_confusion_matrix = confusion_matrix(train_labels, svm_model.predict(reshaped_features))
training_confusion_matrix

Extract the features from the test set

In [ ]:
test_feature_and_labels_extracted = extract_features(test_data)
test_features = test_feature_and_labels_extracted[0]
test_labels = test_feature_and_labels_extracted[1]

Alternatively, you can load the test data from the npz file. This file was saved by me earlier, and contains all of the preprocessed data

In [ ]:
test_loaded_data = np.load('test_data_working.npz')
reshaped_test_features = test_loaded_data['features']
test_labels = loaded_data['labels']

Reshaping the test data

In [ ]:
num_samples, num_channels, height, width = test_features.shape
reshaped_test_features = test_features.reshape(num_samples * num_channels, -1)
test_labels = test_labels.ravel()
test_labels.shape, reshaped_test_features.shape, type(test_labels), type(reshaped_test_features)

Saving the test dataset

In [ ]:
np.savez('test_data_working.npz', features=reshaped_test_features, labels=test_labels)

Finally, scoring the model

In [ ]:
svm_model.score(reshaped_test_features, test_labels)

My final score was typically around 40-50%